<a href="https://colab.research.google.com/github/Gabriel-Machado-GM/Online-Judge-Solutions-Python/blob/main/uva_793_network_connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**@PDF: [UVA - 793 Network Connections](https://onlinejudge.org/external/7/793.pdf)** \
**@AUTOR: [GABRIEL MACHADO](https://github.com/Gabriel-Machado-GM)** \
**@REPO: [ONLINE JUDGE SOLUTIONS PYTHON](https://github.com/Gabriel-Machado-GM/Online-Judge-Solutions-Python)**

# UVA - 793 Network Connections

Bob, who is a network administrator, supervises a network of computers. He is keeping a log connections between the computers in the network. Each connection is bi-directional. Two computers are interconnected if they are directly connected or if they are interconnected with the same computer. Occasionally, Bob has to decide, quickly, whether two given computers are connected, directly or indirectly, according to the log information.

Write a program which based on information input from a text file counts the number of successful and the number of unsuccessful answers to the questions of the kind:

`is computer, interconnected with computer;?`

## Input

The input begins with a single positive integer on a line by itself, indicating the number of the cases following. This line is followed by a blank line, and there is also a blank line between two consecutive inputs.

For each test case, the input must follow the description below.

1.  The number of computers in the network (a strictly positive integer);
2.  A list of pairs of the form:
    * (a) `c computer, computer,` where `computer,` and `computer` are integers from 1 to `no_of_computers`. A pair of this form shows that `computer` and `computer` get interconnected.
    * (b) `q computer, computer;,` where `computer` and `computer;` are integers from 1 to `no_of_computers`. A pair of this form stands for the question: `is computer interconnected with computer;?`

Each pair is on a separate line. Pairs can appear in any order, regardless of their type. The log is updated after each pair of type (a) and each pair of type (b) is processed according to the current network configuration.

## Output

For each test case, the output must follow the description below. The outputs of two consecutive cases will be separated by a blank line.

The program prints two integer numbers to the standard output on the same line, in the order: `successful answers, unsuccessful answers`, as shown in the sample output.

**Note:**

For example, the first input illustrated in the sample below corresponds to a network of 10 computers and 7 pairs. There are '1' successfully answered questions and '2' unsuccessfully answered questions.


## Sample Input

2\
\
10\
c 1 5\
c 2 7\
q 7 1\
c 3 9\
q 9 6\
c 2 5\
q 7 5\
\
1\
q 1 1\
c 1 1\
q 1 1

## Sample Input
1,2\
2,0\

In [ ]:
# Import necessary modules
from time import time  # To measure execution time (optional, commented out later)
from sys import stdin, stdout # For efficient input/output handling

# Define the Union-Find (Disjoint Set Union - DSU) class
class UF:
  """
  An implementation of the union-find data structure.
  It uses the weighted quick union by rank optimization
  along with path compression for efficiency.
  """

  def __init__(self, N):
    """
    Initialize the Union-Find structure for N items (numbered 0 to N-1).

    Args:
        N: The total number of items (nodes/elements).
    """
    # _id: This list stores the parent of each item. Initially, each item is its own parent.
    # Example: If N=5, _id starts as [0, 1, 2, 3, 4]
    self._id = list(range(N))

    # _count: Keeps track of the number of disjoint sets (components). Initially, each item is in its own set.
    self._count = N

    # _rank: Stores the rank (or height/depth approximation) of the tree rooted at each item.
    # Used for the "union by rank" optimization to keep the trees relatively flat.
    # Starts with all ranks as 0 because each item is a root of a tree of height 0.
    self._rank = [0] * N

  def find(self, p):
    """
    Find the root (representative) of the set containing item p.
    Implements path compression.

    Args:
        p: The item whose set representative we want to find.

    Returns:
        The root/representative of the set containing p.
    """
    # Store the parent list locally for quicker access
    id_list = self._id

    # Traverse up the tree until we find the root (an item that is its own parent)
    root = p
    while root != id_list[root]:
        root = id_list[root]

    # Path Compression: Make every node on the path point directly to the root.
    # This flattens the tree significantly for future find operations.
    while p != root:
        next_p = id_list[p] # Store the next node in the original path
        id_list[p] = root   # Point the current node 'p' directly to the root
        p = next_p          # Move to the next node in the original path

    return root # Return the identified root of the set

  def connected(self, p, q):
    """
    Check if items p and q belong to the same set (component).

    Args:
        p: The first item.
        q: The second item.

    Returns:
        True if p and q are in the same set, False otherwise.
    """
    # Two items are connected if they share the same root/representative.
    return self.find(p) == self.find(q)

  def union(self, p, q):
    """
    Merge (unite) the sets containing items p and q.
    Uses the "union by rank" optimization.

    Args:
        p: An item in the first set.
        q: An item in the second set.
    """
    # Find the roots of the sets containing p and q
    i = self.find(p)
    j = self.find(q)

    # If p and q are already in the same set, do nothing.
    if i == j:
      return

    # Union by Rank: Attach the shorter tree to the root of the taller tree.
    # This helps keep the tree heights low, optimizing future 'find' operations.
    id_list = self._id
    rank_list = self._rank

    if rank_list[i] < rank_list[j]:
      # If tree i is shorter than tree j, make root j the parent of root i.
      id_list[i] = j
      # The rank of j doesn't change.
    elif rank_list[i] > rank_list[j]:
      # If tree j is shorter than tree i, make root i the parent of root j.
      id_list[j] = i
      # The rank of i doesn't change.
    else:
      # If both trees have the same rank, arbitrarily make i the parent of j.
      id_list[j] = i
      # Increment the rank of the new root (i) because the tree height increases by 1.
      rank_list[i] += 1

    # Since we merged two sets, decrease the total count of disjoint sets.
    self._count -= 1


# --- Main execution part ---

# Record the start time (optional, for performance analysis)
# s = time()

# Read the number of test cases
num_tests = int(stdin.readline())
# Read the blank line often present between the number of tests and the first test case
stdin.readline()

# Process each test case
for t in range(num_tests):
  # Read the number of computers (nodes) for this test case.
  # Note: The problem likely uses 1-based indexing, so we create N+1 elements (0 to N)
  # to easily map computer IDs 1 to N to indices 1 to N in our lists. Index 0 might be unused.
  N = int(stdin.readline())

  # Initialize the Union-Find structure for N+1 elements.
  uf = UF(N + 1)

  # Initialize counters for successful ('q' queries where nodes are connected)
  # and unsuccessful ('q' queries where nodes are not connected) queries.
  num_connected = 0
  num_disconnected = 0

  # Read operations until a blank line or end-of-file indicates the end of the test case.
  while True:
    try:
      # Read a line of input and remove leading/trailing whitespace
      line = stdin.readline().strip()
      # If the line is empty, we've reached the end of the input for this test case.
      if not line:
        break

      # Split the line into components (command, node1, node2)
      parts = line.split()
      command = parts[0]
      u = int(parts[1]) # First computer ID
      v = int(parts[2]) # Second computer ID

      # Process the command
      if command == "c":
        # 'c' means "connect": perform a union operation on the sets containing u and v.
        uf.union(u, v)
      elif command == "q":
        # 'q' means "query": check if u and v are connected.
        if uf.connected(u, v):
          # If they are connected, increment the connected counter.
          num_connected += 1
        else:
          # If they are not connected, increment the disconnected counter.
          num_disconnected += 1

    except EOFError:
        # Handle the end-of-file condition if it occurs mid-read
        break

  # Print the results for the current test case.
  # Add a blank line between outputs of different test cases if this is not the first one.
  if t > 0:
    stdout.write("\n")

  # Output the counts of successful and unsuccessful queries, comma-separated.
  stdout.write("{},{}\n".format(num_connected, num_disconnected))

# Record the end time (optional)
e = time()

# Calculate and print the elapsed time (commented out as requested)
# print(e - s)